Por alguna razón, si leemos (read) dos veces el archivo (arx) abierto, la segunda vez nos da un array vacio. 
Asi que mas bien lo leemos y lo cerramos de una.
Al parecer es porque read avanza en el registor, así que si no especificamos el número de bytes o pasos, pus nos leemos todo y quedamos al final. De todas formas por seguridad de cosas que no entiendo, dejemos el close.

In [1]:
arx=open("../ABFiles/Eduardo/C1C31000.DAT", "r")
databin=read(arx)
close(arx)

In [2]:
typeof(databin) # Los datos estan en bytes separados en palabras de 8 bytes.

Array{UInt8,1}

In [3]:
head=databin[1:4]

4-element Array{UInt8,1}:
 0x41
 0x42
 0x46
 0x20

In [4]:
sizeof(Float32)

4

In [5]:
string(head[4]), string(50, base=7), Char(head[4])

("32", "101", ' ')

Lo que descubrió swharden es que el 4 número nos dice si es un abf1 o un abf2. Si el número es un espacio (en ASCII es el número 32) es un abf1. Si es un 50 (ASCII para "2") es un abf2:
 - ascii 32 es espacio -> abf1
 - ascii 50 es 2 -> abf2

In [6]:
convert(Int,databin[4]) #aquí vemos que tipo de archivo es. Curiosamente 0x32 es 50, no 32. ( 3 * 16 +2 )

32

In [7]:
map(Char, head) # and here it is

4-element Array{Char,1}:
 'A'
 'B'
 'F'
 ' '

In [8]:
if( Char(head[4])=='2')
    println( "Tenemos un archivo abf2")
elseif ( Char(head[4])==' ')
    println( "Tenemos un archivo abf1")
else
    println( "no tengo zorra idea de que es esto")
end

Tenemos un archivo abf1


In [9]:
aver=databin[13:16]

4-element Array{UInt8,1}:
 0x16
 0x00
 0x00
 0x00

In [10]:
averviendo=IOBuffer(aver)
read(averviendo, Int32)

22

In [11]:
averviendo=IOBuffer(head)
encabezado=read(averviendo, String)

"ABF "

In [12]:
typeof(encabezado)

String

In [13]:
function readfrombyn(data::Array{UInt8,1}, startbyte::Int, endbyte::Int, typeout::Type, n=1::Int)
    # reads byte data for an array, and interprets as typeout, returnint it.
    # n tells if we want only one pieze of info or an array.
    # String is an exception, typeof can be String and we let n=1
    rango=startbyte:endbyte
    aux=IOBuffer(data[rango])
 
    if n==1   
        rango=startbyte:endbyte
        aux=IOBuffer(data[rango])
        result=read(aux, typeout)
    else
        result=Array{typeout}(undef, n)
        sz=sizeof(typeout)
        for k=1:n
            ranguito=(startbyte+(k-1)*sz):(startbyte+(k)*sz)
            aux=IOBuffer(data[ranguito])
            result[k]=read(aux, typeout)
        end
    end
        
    return result
end

readfrombyn (generic function with 2 methods)

In [14]:
a=readfrombyn(databin, 1, 4, String)
ab=typeof(a)

String

In [15]:
aux=IOBuffer(databin[5:8])
read(aux, Float16)

Float16(1638.0)

In [16]:
ffv = readfrombyn(databin, 5,8, Float16)

Float16(1638.0)

In [17]:
nstyck= readfrombyn(databin, 379, 410, Int16, 16)


16-element Array{Int16,1}:
  0
  1
  2
  3
  4
  5
  6
  7
  8
  9
 10
 11
 12
 13
 14
 15

# Encabezado de un ABF1

mmmm tal vez podemos hacer un programa QUE AUTOMATICE esta conversion!

In [18]:
fFileSignature  = readfrombyn(databin, 1, 4, String )
fFileVersionNumber  = readfrombyn(databin, 5,8, Float16)
nOperationMode  = readfrombyn(databin, 9,10, Int16)
lActualAcqLength  = readfrombyn(databin, 11,14, Int32)
nNumPointsIgnored  = readfrombyn(databin, 15, 16, Int16)
lActualEpisodes  = readfrombyn(databin, 17,24, Int32)
lFileStartTime  = readfrombyn(databin, 25,40, Int32)
lDataSectionPtr  = readfrombyn(databin, 41,44, Int32)
lTagSectionPtr  = readfrombyn(databin, 45,48,  Int32)
lNumTagEntries  = readfrombyn(databin, 49,92, Int32)
lSynchArrayPtr  = readfrombyn(databin, 93, 96, Int32)
lSynchArraySize  = readfrombyn(databin, 97,100, Int32)
nDataFormat  = readfrombyn(databin, 101,120, Int16)
nADCNumChannels  = readfrombyn(databin, 121,122, Int16)
fADCSampleInterval  = readfrombyn(databin, 123, 130, Float16)
fSynchTimeUnit  = readfrombyn(databin, 131,138, Float16)
lNumSamplesPerEpisode  = readfrombyn(databin, 139,142, Int32)
lPreTriggerSamples  = readfrombyn(databin, 143, 146, Int32)
lEpisodesPerRun  = readfrombyn(databin, 147, 150, Int32)
fADCRange  = readfrombyn(databin, 245,252, Float16)
lADCResolution  = readfrombyn(databin, 252,255, Int32)
nFileStartMillisecs  = readfrombyn(databin, 367,368, Int16)
nADCPtoLChannelMap = readfrombyn(databin, 379, 410, Int16,16)
nADCSamplingSeq = readfrombyn(databin, 411, 442, Int16,16)

16-element Array{Int16,1}:
  2
  3
 -1
 -1
 -1
 -1
 -1
 -1
 -1
 -1
 -1
 -1
 -1
 -1
 -1
 -1

355

Dado que String es una excepcion, para las proximas dos lecturas vamos a tener que hacer algo rarito:
dividir en cachos el endiablado lector.
Un **byte** es un **byte** es un **byte**.

In [19]:
sA=Array{String}(undef, 16)

16-element Array{String,1}:
 #undef
 #undef
 #undef
 #undef
 #undef
 #undef
 #undef
 #undef
 #undef
 #undef
 #undef
 #undef
 #undef
 #undef
 #undef
 #undef

In [20]:
sADCChannelName=Array{String}(undef, 16)
ini=443
for j=0:15
    inif=ini+10*j
    finf=ini+10*(j+1)-1
    sADCChannelName[j+1]=readfrombyn(databin, inif, finf, String)
end

sADCUnits=Array{String}(undef, 16)
ini=602
for j=0:15
    inif=ini+8*j
    finf=ini+8*(j+1)-1
    sADCUnits[j+1]=readfrombyn(databin, inif, finf, String)
end



Por recordatorio, esta era la definicion en python. El Metodo Karel parece funcionar.
#=
sADCChannelName = readfrombyn(f, "10s"*16, 442)
sADCUnits = readfrombyn(f, "8s"*16, 602)

=#

In [21]:
fADCProgrammableGain = readfrombyn(databin, 731,922, Float16, 16)

16-element Array{Float16,1}:
 0.0  
 2.5  
 0.0  
 2.5  
 0.0  
 2.5  
 0.0  
 2.5  
 0.0  
 1.875
 0.0  
 1.875
 0.0  
 1.875
 0.0  
 1.875

In [22]:
fInstrument= readfrombyn(databin,923,986,  Float16 , 16)
fInstrumentOffset = readfrombyn(databin, 987,1050, Float16, 16)

16-element Array{Float16,1}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [23]:
fSignalGain = readfrombyn(databin, 1051, 1114, Float16, 16)
fSignalOffset = readfrombyn(databin, 1115, 1436, Float16, 16)
nDigitalEnable = readfrombyn(databin, 1437,1440, Int16)

1

In [24]:
nActiveDACChannel = readfrombyn(databin, 1441, 1442, Int16)
nDigitalHolding = readfrombyn(databin, 1585, 1586, Int16)
nDigitalInterEpisode = readfrombyn(databin, 1587,1588,Int16)
nDigitalValue = readfrombyn(databin, 2589, 2608, Int16,10)

10-element Array{Int16,1}:
 -192
   64
 -176
   80
 -176
   80
 -176
   80
 -176
   96

In [30]:
nADCNumChannels

2

In [25]:
lDACFilePtr = readfrombyn(databin, 2049,2056, Int32, 2)
lDACFileNumEpisodes = readfrombyn(databin, 2057, 2060, Int32,2)
fDACCalibrationFactor = readfrombyn(databin,  2075, 2090, Float16, 4)
fDACCalibrationOffset = readfrombyn(databin,  2091, 2105, Float16,4)
nWaveformEnable = readfrombyn(databin, 2297,2300, Int16,2)
nWaveformSource = readfrombyn(databin,  2301,2304, Int16,2)
nInterEpisodeLevel = readfrombyn(databin,  2305,2308, Int16, 2)
nEpochType = readfrombyn(databin, 2309,2348, Int16,20)
fEpochInitLevel = readfrombyn(databin, 2349, 2428, Float16, 20)
fEpochLevelInc = readfrombyn(databin,  2429, 2488, Float16,20)
lEpochInitDuration = readfrombyn(databin,  2509, 2588,Int32,20)
lEpochDurationInc = readfrombyn(databin,  2589, 2668,  Int32,20)
nTelegraphEnable = readfrombyn(databin,  4513,4544, Int16, 16)
fTelegraphAdditGain = readfrombyn(databin,  4577, 4609, Float16,16)
#sProtocolPath = readfrombyn(databin,  4898,5282 , String)

16-element Array{Float16,1}:
 NaN     
   9.5e-7
 NaN     
   1.9e-6
 NaN     
   9.5e-7
 NaN     
   1.9e-6
 NaN     
   9.5e-7
 NaN     
   9.5e-7
 NaN     
   0.0   
 NaN     
   9.5e-7

In [27]:
nADCPtoLChannelMap

16-element Array{Int16,1}:
  0
  1
  2
  3
  4
  5
  6
  7
  8
  9
 10
 11
 12
 13
 14
 15

In [81]:
typeof(sProtocolPath)

String

In [82]:
collect(sProtocolPath)

385-element Array{Char,1}:
 '\xff'
 '\0'  
 '\0'  
 '@'   
 '\xff'
 '\x10'
 '\0'  
 'P'   
 '\xff'
 '\x10'
 '\0'  
 'P'   
 '\xff'
 ⋮     
 ' '   
 '\0'  
 ' '   
 '\xff'
 ' '   
 '\0'  
 '0'   
 '\xff'
 ' '   
 '\0'  
 '0'   
 '\xff'